In [79]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import os
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader

# Load data

In [80]:
transform = transforms.Compose([
    transforms.ToTensor()
])

### Load data-figure out paths

In [81]:
rendered_path = "/Users/wenruili/Documents/Computational periscopy/testresized"
gt_path = "/Users/wenruili/Documents/Computational periscopy/gt"
rendered_file = [os.path.join(rendered_path,i) for i in os.listdir(rendered_path)]
gt_file = [os.path.join(gt_path,i) for i in os.listdir(gt_path)]
np.save("rendered_file.npy",rendered_file)
np.save( "gt_file.npy",gt_file)

### Load data - config dataloader and load

In [84]:
def rendered_loader(path):
    img_pil = Image.open(path)
    img_pil = img_pil.resize((224,224))
    img_tensor = transform(img_pil)
    return img_tensor
def gt_loader(path):
    img_pil = Image.open(path)
    img_pil = img_pil.resize((36,29))
    img_tensor = transform(img_pil)
    return img_tensor
class trainset(Dataset):
    def __init__(self, render_loader = rendered_loader, gt_loader = gt_loader):
        self.rendered = rendered_file
        self.gt = gt_file
        self.render_loader = rendered_loader
        self.gt_loader = gt_loader
    def __getitem__(self,index):
        rendered = self.render_loader(self.rendered[index])
        gt = self.gt_loader(self.gt[index])
        return rendered,gt
    def __len__(self):
        return len(self.rendered)
train_data = trainset()
trainloader = DataLoader(train_data,batch_size = 1,shuffle = True)

In [85]:
def show_batch():
    for epoch in range(3):
        print("epoch#####################",epoch)
        for step, (batch_x, batch_y) in enumerate(trainloader):
            print(step,batch_x.shape,batch_y.shape)
            # training
            #print("steop:{}, batch_x:{}, batch_y:{}".format(step, batch_x, batch_y))
show_batch()

epoch##################### 0
0 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
1 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
2 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
3 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
4 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
5 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
6 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
7 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
epoch##################### 1
0 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
1 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
2 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
3 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
4 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
5 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
6 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
7 torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 29, 36])
epoch#########

## Getting ready to train
rendered_data.shape should be [batch,1,126,126]

gt.shape should be [1,36,29]

Also need to load matrix A , A.shape = [15876,1044]

Loss function: 
$$ \min_{A, CNN} \sum_i \| A CNN(x_i) - x_i \|^2 + \beta\|CNN(x_i) - y_i\|^2+ \lambda \| A - \hat{A} \|^2$$

In [87]:
from module import AlexNet
from scipy.io import loadmat
from torch.autograd import Variable
m = loadmat("sima.mat")
simA = m["simA"]
simA = torch.Tensor(simA)
simA.shape


torch.Size([15876, 1044])

In [77]:
A = Variable(torch.rand([15876, 1044])*40)
model = AlexNet()
print(model)
model.forward(torch.randn(1,3,126,126))

AlexNet(
  (feature_extraction): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2), bias=False)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features

RuntimeError: shape '[1, 9216]' is invalid for input of size 1024

In [78]:
CNN_output = torch.randn(10,1044)
simA = torch.randn(15876,1044)
y = torch.randn(10,1,126,126)
batch_size = 9
loss = 0
range(batch_size)
for i in range(batch_size):
    tmp = torch.matmul(simA,CNN_output[batch_size,:]) - y[batch_size,:].view(15876)
    loss += torch.norm(tmp,p = 2, dim = 0)
    print(loss)
torch.matmul(simA,CNN_output[batch_size,:]).shape

def loss_function(preds,targets,A,simA,lamda,batchsize):
    for i in range(batch_size):
    tmp = torch.matmul(simA,preds[batch_size,:]) - y[batch_size,:].view(15876)
    loss += torch.norm(tmp,p = 2, dim = 0)
    print(loss)
    loss = torch.sum(A)
    

tensor(3989.4998)
tensor(7978.9995)
tensor(11968.4990)
tensor(15957.9990)
tensor(19947.4980)
tensor(23936.9980)
tensor(27926.4980)
tensor(31915.9980)
tensor(35905.4961)


torch.Size([15876])

In [ ]:
# for epoch in range(1000):
#         t = time.time()
#         model.train()
#         optimizer.zero_grad()
#         recovered, mu, logvar = model(features, adj_norm)
#         loss = loss_function(preds=recovered, labels=adj_label,
#                              mu=mu, logvar=logvar, n_nodes=n_nodes,
#                              norm=norm, pos_weight=pos_weight)
#         loss.backward()
#         cur_loss = loss.item()
#         optimizer.step()

#         hidden_emb = mu.data.numpy()
#         roc_curr, ap_curr = get_roc_score(hidden_emb, adj_orig, val_edges, val_edges_false)

#         print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss),
#               "val_ap=", "{:.5f}".format(ap_curr),
#               "time=", "{:.5f}".format(time.time() - t)
#               )
